## Convolutional Neural Network

In [147]:
import tensorflow as tf
import numpy as np

In [148]:
(X_train, y_train),(X_test,y_test)= tf.keras.datasets.mnist.load_data()

In [149]:
X_train = X_train.astype(np.float32).reshape(-1,28*28)/255.0      #Change colour to intensity
X_test = X_test.astype(np.float32).reshape(-1,28*28)/255.0

In [150]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

### Some constant declaration

In [151]:
height,width = 28, 28
#channels is used to make #rd dimentional as 1
channels = 1
n_inputs = height*width
n_output = 10

### CNN = 2CL + 1PL + 1FCL + 10pL

In [152]:
#Constants for CL 1
conv1_fmaps =  32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = 'SAME'

In [153]:
# Constants for CL 2
conv2_fmaps =  64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = 'SAME'

In [154]:
# Constants for PL
pool_fmaps = conv2_fmaps
pool_ksize = [1,2,2,1]
pool_stride = [1,2,2,1]
pool_pad = 'VALID'

## Create Layers

In [155]:
tf.reset_default_graph()

In [156]:
X = tf.placeholder(dtype=tf.float32,shape=[None,n_inputs],name = 'x')

In [157]:
X_reshape = tf.reshape(X,shape = [-1,height,width,channels])     #Actual input layers

In [158]:
y = tf.placeholder(dtype=tf.int32,shape=[None],name = 'y')

## Convolution layer

In [159]:
convo1 = tf.layers.conv2d(inputs=X_reshape,
                           filters=conv1_fmaps,
                           kernel_size=conv1_ksize,
                           strides=conv1_stride,
                           padding=conv1_pad,
                           activation=tf.nn.relu,
                           name='conv1')

In [160]:
convo2 = tf.layers.conv2d(inputs=X_reshape,
                          filters=conv2_fmaps,
                          kernel_size=conv2_ksize,
                          strides=conv2_stride,
                          padding=conv2_pad,
                          activation=tf.nn.relu,
                          name='conv2')

In [161]:
pool = tf.nn.max_pool(value = convo2,ksize=pool_ksize,strides=pool_stride,padding=pool_pad,name='pool')

In [162]:
pool_flatten = tf.reshape(pool,shape=[-1,7*7*pool_fmaps],name='pool_flatten')       #All must be factors of original size

### Fully Connected Layer

In [163]:
fc = tf.layers.dense(pool_flatten,units=64,activation = tf.nn.relu,name='fc')

### Output Layer

In [164]:
output_logits = tf.layers.dense(fc,units=n_output,activation=None,name='out_logits')

### Prediction

In [165]:
y_pred = tf.nn.softmax(output_logits)

#### Cross entropy and otimization

In [166]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output_logits,labels=y)

In [167]:
loss = tf.reduce_mean(xentropy)

### Using Adam Optimizer

In [168]:
optimizer = tf.train.AdamOptimizer()

In [169]:
optimizer = optimizer.minimize(loss)

#### Metrics

In [170]:
correct = tf.nn.in_top_k(output_logits,y,1)

In [171]:
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

### Initialize all variables asnd saver NODES

In [172]:
init = tf.global_variables_initializer()

In [173]:
saver = tf.train.Saver()

### Create and run it in a session

In [175]:
n_epoch = 2
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        sess.run(optimizer,feed_dict={X:X_train[:60],y:y_train[:60]})
        print(accuracy.eval(feed_dict={X:X_test[:6],y:y_test[:6]}))
    pred = sess.run(y_pred,feed_dict={X:X_test[0:2]})    

0.16666667
0.33333334


In [176]:
list(pred[0]).index(np.max(pred[0])),list(pred[1]).index(np.max(pred[1]))

(7, 3)

In [177]:
y_test[0:2]

array([7, 2])

### Recurrent Neural Network

In [205]:
import tensorflow as tf
import numpy as np

In [206]:
tf.reset_default_graph()

In [207]:
X0 = tf.placeholder(dtype=tf.float32,shape=[None,3],name='X0')
X1 = tf.placeholder(dtype=tf.float32,shape=[None,3],name='X1')

### Creating RNN call and then create RNN

In [208]:
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = 3)

In [209]:
outputs,states = tf.nn.static_rnn(cell=basic_cell,inputs=[X0,X1],dtype = tf.float32)

In [210]:
Y0,Y1 = outputs

### Create some random data for operation

In [211]:
X0_data = np.array([[1,2,3],[4,5,6],[7,8,9]])
X1_data = np.array([[11,22,33],[44,55,66],[77,88,99]])

In [212]:
init = tf.global_variables_initializer()

In [213]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0,Y1],feed_dict = {X0:X0_data,X1:X1_data})

In [214]:
Y0_val , Y1_val

(array([[-0.97834975,  0.06927356, -0.9674872 ],
        [-0.9992668 , -0.02165861, -0.99998254],
        [-0.9999755 , -0.11223396, -1.        ]], dtype=float32),
 array([[-1.        ,  0.8719821 , -1.        ],
        [-1.        ,  0.3692143 , -1.        ],
        [-1.        , -0.52134126, -1.        ]], dtype=float32))

### USing RNN fro MNIST

In [217]:
(X_train, y_train) , (X_test,y_test) = tf.keras.datasets.mnist.load_data()

In [245]:
X_train = X_train.astype(np.float32).reshape(-1,28,28)/255.0      #Change colour to intensity
X_test = X_test.astype(np.float32).reshape(-1,28,28)/255.0

In [246]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#### Declare some constants

In [247]:
n_output = 10
n_neurons = 10
n_layer = 3
height, width = 28,28

#### Input layer

In [254]:
tf.reset_default_graph()

In [255]:
X = tf.placeholder(dtype=tf.float32,shape=[None,height,width],name = 'x')

In [256]:
y = tf.placeholder(dtype=tf.int32,shape=[None],name='y')

#### Multi Layer RNN
    * create a list of RNN cells
    
##### Syntax for FOR-loop inside a loop
    some_list = [
                [operation]
                for iteratable in iteratable_object   
    ]

In [257]:
list_rnn_cells = [
    tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons,activation=tf.nn.relu)
    for it in range(n_layer)
]

In [258]:
multi_layer_rnn = tf.nn.rnn_cell.MultiRNNCell(list_rnn_cells)

In [259]:
outputs, states = tf.nn.dynamic_rnn(cell=multi_layer_rnn,inputs=X,dtype=tf.float32)

In [260]:
outputs, states       # states contains all the data (metadata ,etc.)

(<tf.Tensor 'rnn/transpose_1:0' shape=(?, 28, 10) dtype=float32>,
 (<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 10) dtype=float32>,
  <tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 10) dtype=float32>,
  <tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 10) dtype=float32>))

In [261]:
states_concat = tf.concat(values = states, axis=1)

#### Create a output layer connected to all the steps in the Time Step

In [262]:
output_logits = tf.layers.dense(inputs=states_concat, units=n_output,activation=None)

#### Optimizer for the model

In [263]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output_logits,labels = y)

In [264]:
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer()


In [265]:
optimizer = optimizer.minimize(loss)

In [266]:
correct = tf.nn.in_top_k(output_logits,y,1)

In [267]:
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

In [268]:
init = tf.global_variables_initializer()

In [278]:
with tf.Session() as sess:
    init.run()
    for epoch  in range(1):
        sess.run(
            optimizer,
            feed_dict={X:X_train[epoch*100:(epoch+10)*100],y:y_train[epoch*100:(epoch+10)*100]})
        print(accuracy.eval(feed_dict={X:X_test[epoch*100:(epoch+1)*100],y:y_test[epoch*100:(epoch+1)*100]}))

0.1
